In [ ]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

from pprint import pprint as pprint

from scipy.optimize import minimize

import os, sys

In [ ]:
# base of configurations
# reference: http://www.gwoptics.org/finesse/examples/aligo_sensitivity.php

basekat = finesse.kat()
basecode = """
# lasers
const Pin 125
const PLO1 1
# spaces
const lx 1.6
const ly 1.55
const Lx 4k
const Ly 4k
const lprc 53
# mirrors
const Tprm 0.03
const Loss_prm 20E-6 # 1E-3
const Titm 0.014
const Loss_itm 20E-6 # 37.5E-6
const Tetm 0 # 6E-6
const Loss_etm 20E-6 # 37.5E-6
const Tsrm 0.2
const Loss_srm 20E-6 # 35E-6
const Mtm 40 #kg

# carrier laser
l laser $Pin 0 nLaser

# power recycling
m1 PRM $Tprm $Loss_prm 90 nLaser nPRM # tuning = 90° towards nLaser
s sBStoPRM $lprc nPRM nBSi

# central beam splitter
bs BS 0.5 0.5 0 45 nBSi nBSr nBSt nBSo
s sBStoYarm $ly nBSr nY0
s sBStoXarm $lx nBSt nX0

# Y arm (perpendicular)
m1 mYitm $Titm $Loss_itm 0 nY0 nY1
attr mYitm mass $Mtm
s sY $Ly nY1 nY2
m1 mYetm $Tetm $Loss_etm 0 nY2 nY3
attr mYetm mass $Mtm

# X arm (parallel)
m1 mXitm $Titm $Loss_itm 90 nX0 nX1 # tuning = 90° towards nX0
attr mXitm mass $Mtm
s sX $Lx nX1 nX2
m1 mXetm $Tetm $Loss_etm 90 nX2 nX3 # tuning = 90° towards nX2
attr mXetm mass $Mtm

# signal recycling
# nBSo to nSRM left disconnected for now
# is connected differently in kat1 and kat2
m1 SRM $Tsrm $Loss_srm 90 nSRM nDark
s sSRMtohdBS 0 nDark nhdBSi # null length space

# (balanced) homodyne detection
# phase of LO s.t. beamsplitter has one constructive, one destructive
l LO1 $PLO1 0 -90 nLO1
bs hdBS 0.5 0.5 0 45 nhdBSi nhdBSr nhdBSt nLO1
qhd qhd1 180 nhdBSr nhdBSt
hd hd1 180 nhdBSr nhdBSt
"""
basekat.parse(basecode)

# pprint((basekat.components, basekat.detectors, basekat.commands))

In [ ]:
def check_tuning(kat, power_check=False):
    """checking IFO tuning by seeing if circulating power in arms is close to 800kW"""
    check_tuning_kat = deepcopy(kat)
    check_tuning_kat.verbose = False
    check_tuning_code="""
    pd initial nLaser*
    pd circPRC nPRM
    pd circX nX1
    pd circY nY1

    pd BSr nBSr
    pd BSt nBSt
    pd BSo nBSo

    noxaxis
    """
    check_tuning_kat.parse(check_tuning_code)
    check_tuning_out = check_tuning_kat.run()
    
    if power_check:
        pprint([(x, '{0:.2f}'.format(check_tuning_out[x].mean())) for x in
                ["initial","circPRC","circX","BSt"]])         
        print("power ratio: arms/(half of BS) =  {0:.20g}".format(
            check_tuning_out['circX']/check_tuning_out['BSt']))
        print()
    else:
        pprint([(x, '{0:.2f}'.format(check_tuning_out[x].mean())) for x in check_tuning_out.ylabels]) 

In [ ]:
class HiddenPrints:
    """hides prints in block when used as "with HiddenPrints():"
    https://stackoverflow.com/a/45669280
    """
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [ ]:
def create_kat1(lsrc):
    """configuration 1: connecting nBSo and nSRM with a space
    lsrc in m
    """
    kat1 = deepcopy(basekat)
    kat1code = """
    const lsrc {0}
    s sBStoSRM $lsrc nBSo nSRM
    """.format(lsrc)
    kat1.parse(kat1code)
    
    return kat1


def create_kat2(lsrc, sqz_gain=0.1, sqz_angle=0):
    """configuration 2: connecting nBSo and nSRM with a nle
    lsrc in m, sqz_gain in field dB, sqz_angle in deg
    """
    kat2 = deepcopy(basekat)
    kat2code = """
    const lsrc {0}
    const halflsrc {1}
    s sBStonle $halflsrc nBSo nnle11
    nle nle1 {2} {3} nnle11 nnle12
    s snletoSRM $halflsrc nnle12 nSRM  
    """.format(lsrc, lsrc/2, sqz_gain, sqz_angle)
    with HiddenPrints():
        kat2.parse(kat2code)
    
    return kat2

In [ ]:
def differential_transfer(ifo, fmin=None, fmax=None):
    """calculates transfer function for ifo object of kat1 or kat2
    for differential shaking of arm spaces
    NB: fsig accounts for vacuum noise
    """
    kat = ifo.kat
    if fmin is None:
        fmin = ifo.fmin
    if fmax is None:
        fmax = ifo.fmax    
    
    transfer_kat = deepcopy(kat)
    transfer_kat.verbose = False
    # initial frequency of 1 for fsig doesn't matter   
    transfer_code = """          
    fsig darm sX 1 0
    fsig darm sY 1 180 # differential tuning
    
    xaxis darm f log {0} {1} 100
    yaxis log abs
    """.format(fmin, fmax)
    transfer_kat.parse(transfer_code)
    transfer_out = transfer_kat.run()    
    
    return transfer_out

In [ ]:
class IFO(object):
    def __init__(self, include_nle, lsrc, title=None, filetag=None,
                 fmin=10, fmax=1e4, sqz_gain=0.1, sqz_angle=0, homodyne_angle=None,
                 phi_srm=None, T_srm=None, T_prm=None, P_in=None):
        """instance of kat1 or kat2
        include_nle is flag for whether to include nle in config
        lsrc is space from central BS to SRM
        title is appended to all plot titles
        filetag is appended to all filenames, no .pdf required
        (fmin, fmax) is frequency range for xaxis
        sqz_gain, sqz_angle are values for nle
        homodyne_angle is set after creation if specified
        """
        self.lsrc = lsrc
        if title is None:
            self.title = repr((include_nle, lsrc))        
        else:
            self.title = title
        if filetag is None:
            self.filetag = ".pdf"
        else:
            self.filetag = filetag+".pdf"
        self.fmin = fmin
        self.fmax = fmax
        
        if not include_nle:
            self.kat = create_kat1(lsrc)
        elif include_nle:
            self.kat = create_kat2(lsrc, sqz_gain, sqz_angle)
        else:
            raise ValueError
            
        if homodyne_angle is not None:
            self.kat.LO1.phase = homodyne_angle
            
        if T_srm is not None:
            self.kat.SRM.T = T_srm
            self.kat.SRM.R = 1 - T_srm - self.kat.SRM.L.value
            
        if T_prm is not None:
            self.kat.PRM.T = T_prm
            self.kat.PRM.R = 1 - T_prm - self.kat.PRM.L.value
            
        if phi_srm is not None:
            self.kat.SRM.phi = phi_srm
            
        if P_in is not None:
            self.kat.laser.P = P_in            
            
    def plot_transfer_fns_and_sensitivity(self, fmin=None, fmax=None, show=True):
        """plot transfer functions and noise limited sensitivity"""
        if fmin is None:
            fmin = self.fmin
        if fmax is None:
            fmax = self.fmax
        
        plot_out = differential_transfer(self)
        plot_x = plot_out.x
        plot_noise = plot_out['qhd1']
        plot_signal = plot_out['hd1']

        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8))

        ax1.plot(plot_x, plot_noise, label="qhd1")
        ax1.set(title="noise transfer fn - "+self.title,
                xlabel="frequency / Hz", ylabel="noise / au",
                yscale="log", xscale="log")
        ax1.legend()

        ax2.plot(plot_x, plot_signal, label="hd1")
        ax2.set(title="signal transfer fn - "+self.title,
                xlabel="frequency / Hz", ylabel="signal / au",
                yscale="log", xscale="log")
        ax2.legend()

        ax3.plot(plot_x, plot_noise/plot_signal, label="NSR")
        ax3.set(title="QN limited sensitivity - "+self.title,
                xlabel="frequency / Hz", ylabel="strain / ${Hz}^{-1/2}$",
                yscale="log", xscale="log")
        ax3.legend()

        fig.tight_layout()
        fig.savefig("aLIGO_transfer_fns_and_sensitivity"+self.filetag)             

In [ ]:
def combined_sensitivity_plots(filename, *args):
    """combines sensitivity plots for ifo's over same frequency axis
    expects each arg in args to be of the form (ifo, label, fmt)
    if no preference to format, input empty string
    """
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8,8), sharex=True)
    
    plot_x = None
    
    for arg in args:
        ifo, label, fmt = arg

        out = differential_transfer(ifo)
        noise = out['qhd1']
        signal = out['hd1']
        if plot_x is None:
            plot_x = out.x
            
        ax1.plot(plot_x, noise, fmt, label="qhd1-"+label)
        ax2.plot(plot_x, signal, fmt, label="hd1-"+label)
        ax3.plot(plot_x, noise/signal, fmt, label="NSR-"+label)
        
    ax1.set(title="noise transfer fn", ylabel="noise",
            yscale="log", xscale="log")   
    ax2.set(title="signal transfer fn", ylabel="signal",
            yscale="log", xscale="log")
    ax3.set(title="QN limited sensitivity",
            xlabel="frequency / Hz", ylabel="strain / ${Hz}^{-1/2}$",
            yscale="log", xscale="log")
    ax1.legend()     
    ax2.legend()
    ax3.legend()
    
    fig.tight_layout()
    fig.savefig(filename)

In [ ]:
def peak_sensitivity(ifo, fmin=0, fmax=1e10):
    """given an ifo, return peak sensitivity and frequency thereof
    peak is only taken within fmin to fmax
    """
    opt_out = differential_transfer(ifo)
    opt_nsr = opt_out['qhd1']/opt_out['hd1']

    fmin_index = opt_out.x.searchsorted(fmin)
    fmax_index = opt_out.x.searchsorted(fmax)
    opt_nsr_cut = opt_nsr[fmin_index:fmax_index]

    peak_depth = opt_nsr_cut.min()
    peak_freq = (opt_out.x[fmin_index:fmax_index])[opt_nsr_cut.argmin()]

    return peak_depth, peak_freq

def ifo_args_to_sensitivity(lsrc, sqz_gain, sqz_angle, homodyne_angle, fmin=0, fmax=1e10):
    """given squeezer gain, squeezer angle, and homodyne angle, return peak sensitivity
    peak is only taken within fmin to fmax    
    """
    ifo = IFO(1, lsrc, sqz_gain=sqz_gain, sqz_angle=sqz_angle, homodyne_angle=homodyne_angle)
    
    return peak_sensitivity(ifo, fmin=fmin, fmax=fmax)

def best_peak_sensitivity(lsrc, init_sqz_gain, init_sqz_angle, init_homodyne_angle,
                          fmin=0, fmax=1e10,
                          return_values=True, print_values=False):
    """optimise depth of sensitivity at cavity pole for given lsrc
    against squeezer gain, squeezer angle, and homodyne angle
    returns best peak sensitivity and arguments thereof
    peak is only taken within fmin to fmax    
    """
    log_opt_fn = lambda x, lsrc, fmin, fmax: np.log(ifo_args_to_sensitivity(lsrc, x[0], x[1], x[2],
                                                                fmin=fmin, fmax=fmax)[0])

    with HiddenPrints():
        result = minimize(log_opt_fn, (init_sqz_gain, init_sqz_angle, init_homodyne_angle),
                          args=(lsrc, fmin, fmax),
                          bounds=((0, None), (-180, 180), (-180, 180)))
    
    if not result["success"]:
        raise ValueError("minimisation didn't converge")
    
    sensitivity = np.exp(result['fun'])
    sensi_args = result['x']
    
    if print_values:
        print("""
for SRC of length: {4}
local best peak sensitivity: {0:.4g}
within frequency range of: {5:.4g} to {6:.4g}
obtained for:
    squeezer gain = {1:.3g}
    squeezer angle = {2:.3g}
    homodyne angle = {3:.3g}
        """.format(sensitivity, *sensi_args, lsrc, fmin, fmax))
    
    if return_values:
        return np.exp(result['fun']), result['x']

In [ ]:
def ifo_args_to_sensitivity_2(lsrc, sqz_gain, sqz_angle, homodyne_angle,
                              T_srm, phi_srm, fmin=0, fmax=1e10):
    """given ifo parameters, return peak sensitivity
    peak is only taken within fmin to fmax    
    """
    ifo = IFO(1, lsrc, sqz_gain=sqz_gain, sqz_angle=sqz_angle,
              homodyne_angle=homodyne_angle, T_srm=T_srm, phi_srm=phi_srm)
    
    return peak_sensitivity(ifo, fmin=fmin, fmax=fmax)

def best_peak_sensitivity_2(lsrc_i, sqz_gain_i, sqz_angle_i, homodyne_angle_i, T_srm_i, phi_srm_i,
                            fmin=0, fmax=1e10, return_values=True, print_values=False, param_bounds=None):
    """optimise depth of sensitivity at cavity pole
    against squeezer gain, squeezer angle, homodyne angle,
    length of SRC, tuning of SRC, and power transmission of SRM
    given initial values for each
    returns best peak sensitivity and arguments thereof
    peak is only taken within fmin to fmax
    """
    # x = lsrc, sqz_gain, sqz_angle, homodyne_angle, T_srm, phi_srm
    log_opt_fn = lambda x, fmin, fmax: np.log(ifo_args_to_sensitivity_2(*x, fmin=fmin, fmax=fmax)[0])

    initial_values = (lsrc_i, sqz_gain_i, sqz_angle_i, homodyne_angle_i, T_srm_i, phi_srm_i)
    # revise sqz_gain bounds
    if param_bounds is None:
        param_bounds = ((0, None), (0, None), (-180, 180), (-180, 180), (1e-5, 1-1e-5), (-180, 180))
    
    result = minimize(log_opt_fn, initial_values, args=(fmin, fmax), bounds=param_bounds)
    
    if not result["success"]:
        raise ValueError("minimisation didn't converge")
    
    sensitivity = np.exp(result['fun'])
    opt_params = result['x']
    
    if print_values:
        print("""local best peak sensitivity: {0:.4g}
within frequency range of: {1:.4g} to {2:.4g}
obtained for:
    length of SRC = {3:.3g}
    squeezer gain = {4:.3g}
    squeezer angle = {5:.3g}
    homodyne angle = {6:.3g}
    power transmission of SRM = {7:.3g}
    tuning of SRM = {8:.3g}""".format(sensitivity, fmin, fmax, *opt_params))
    
    if return_values:
        return np.exp(result['fun']), result['x']

In [ ]:
def adjust_losses(kat, Loss_itm=None, Loss_etm=None, Loss_prm=None, Loss_srm=None):
    """updates L and R in kat, leaves T untouched"""
    if Loss_itm is not None:
        kat.mXitm.L = Loss_itm
        kat.mXitm.R = 1 - kat.mXitm.T.value - Loss_itm    
        kat.mYitm.L = Loss_itm
        kat.mYitm.R = 1 - kat.mYitm.T.value - Loss_itm            
        
    if Loss_etm is not None:
        kat.mXetm.L = Loss_etm
        kat.mXetm.R = 1 - kat.mXetm.T.value - Loss_etm    
        kat.mYetm.L = Loss_etm
        kat.mYetm.R = 1 - kat.mYetm.T.value - Loss_etm           
    
    if Loss_prm is not None:
        kat.PRM.L = Loss_prm
        kat.PRM.R = 1 - kat.PRM.T.value - Loss_prm
        
    if Loss_srm is not None:
        kat.SRM.L = Loss_srm
        kat.SRM.R = 1 - kat.SRM.T.value - Loss_srm                    
        
    if np.any(np.abs(np.array([kat.PRM.R.value, kat.SRM.R.value,
                               kat.mXetm.R.value, kat.mXitm.R.value])) > 1):
        raise ValueError("R + T + L > 1")

In [ ]:
check_tuning(basekat)

In [ ]:
fmin, fmax = 10, 1e4

# k is a poor choice of name, given that these are ifo's, not kat's
k11 = IFO(0, 53,   "config. 1, $l_{src}$ = 53 m", "_k11", fmin, fmax)
k12 = IFO(0, 2000, "config. 1, $l_{src}$ = 2 km", "_k12", fmin, fmax)
k21 = IFO(1, 53,   "config. 2, $l_{src}$ = 53 m", "_k21", fmin, fmax)
k22 = IFO(1, 2000, "config. 2, $l_{src}$ = 2 km", "_k22", fmin, fmax)

combined_sensitivity_plots("aLIGO_transfer_fns_and_sensitivity_comparison.pdf",
                           (k11, "no nle-short SRC", ""),
                           (k12, "no nle-long SRC", "--"),
                           (k21, "nle-short SRC", "g"),
                           (k22, "nle-long SRC", "--"))

In [ ]:
k21_local_min = best_peak_sensitivity(53, 0.1, 0, -90, print_values=True, fmin=900, fmax=3000)
k22_local_min = best_peak_sensitivity(2000, 0.1, 0, -90, print_values=True, fmin=900, fmax=3000)

# combined plotting, but with locally optimum parameters
# for config. 1 can just use k11 and k12 again
k21_opt = IFO(1, 53, sqz_gain=k21_local_min[1][0],
              sqz_angle=k21_local_min[1][1], homodyne_angle=k21_local_min[1][2])
k22_opt = IFO(1, 2000, sqz_gain=k22_local_min[1][0],
              sqz_angle=k22_local_min[1][1], homodyne_angle=k22_local_min[1][2])

combined_sensitivity_plots("aLIGO_optimum_sensitivity_comparison.pdf",
                           (k11, "no nle-short SRC", ""),
                           (k12, "no nle-long SRC", "--"),
                           (k21_opt, "nle-short SRC-optimised", "c"),
                           (k22_opt, "nle-long SRC-optimised", "--y"))

In [ ]:
# for comparing against analytics
analytics_lengths = 53, 539.75, 1026.5, 1513.25, 2000

# todo to-do: actually use the values from analytics (ask Vaishali)
analytics_sqz_gain = 0.1
analytics_sqz_angle = 0
analytics_homodyne_angle = -90

analytics_ifos = ((IFO(1, l,
                       sqz_gain=analytics_sqz_gain,
                       sqz_angle=analytics_sqz_angle,
                       homodyne_angle=analytics_homodyne_angle),
                   "nle-$l_{{src}}$ = {0:.3f}".format(l),
                   "") for l in analytics_lengths)

combined_sensitivity_plots("aLIGO_sensitivities_to_compare_to_analytics.pdf",
                           *analytics_ifos)

In [ ]:
# comparing to Vaishali's dashed pink internal squeezed NEMO plot
a2_lsrc = 319
a2_sqz_gain = 0.06 # 0.065?
a2_sqz_angle = 0
a2_homodyne_angle = 90
a2_Tsrm = 0.12

a2_fmin = 1e2
a2_fmax = 5e3

a2_ifo_no_nle_ref = IFO(0, a2_lsrc, fmin=a2_fmin, fmax=a2_fmax, T_srm=a2_Tsrm)
a2_ifo_sqz = IFO(1, a2_lsrc, fmin=a2_fmin, fmax=a2_fmax, sqz_gain=a2_sqz_gain, sqz_angle=a2_sqz_angle,
                 homodyne_angle=a2_homodyne_angle, T_srm=a2_Tsrm)

check_tuning(a2_ifo_sqz.kat)

combined_sensitivity_plots("aLIGO_pink_analytics_comparison.pdf",
                           (a2_ifo_sqz, "nle-$L_{{src}}$ = {0:.0f}".format(a2_lsrc), "--m"),
                           (a2_ifo_no_nle_ref, "no nle-$L_{{src}}$ = {0:.0f}".format(a2_lsrc), ""))

In [ ]:
# optimising over six parameters now
# initial values
lsrc_i, sqz_gain_i, sqz_angle_i, homodyne_angle_i, T_srm_i, phi_srm_i = (319, 0.005, 0, 90, 0.12, 90)
opt_2_result = best_peak_sensitivity_2(lsrc_i, sqz_gain_i, sqz_angle_i,
                                       homodyne_angle_i, T_srm_i, phi_srm_i,
                                       print_values=True, fmin=900, fmax=3000,
                                       param_bounds=((0, None), (1e-3, 1e-2), (-180, 180),
                                                     (0, 180), (1e-5, 1-1e-5), (0, 90)))

opt_2_plot_fmin = 1e2
opt_2_plot_fmax = 5e3

opt_2_ifo = IFO(1, lsrc=opt_2_result[1][0], sqz_gain=opt_2_result[1][1],
                sqz_angle=opt_2_result[1][2], homodyne_angle=opt_2_result[1][3],
                T_srm=opt_2_result[1][4], phi_srm=opt_2_result[1][5],
                fmin=opt_2_plot_fmin, fmax=opt_2_plot_fmax)
opt_2_ifo_ref = IFO(1, lsrc=lsrc_i, sqz_gain=sqz_gain_i,
                    sqz_angle=sqz_angle_i, homodyne_angle=homodyne_angle_i,
                    T_srm=T_srm_i, phi_srm=phi_srm_i,
                    fmin=opt_2_plot_fmin, fmax=opt_2_plot_fmax)

combined_sensitivity_plots("aLIGO_six_param_optimisation.pdf",
                           (opt_2_ifo, "nle-optimised", ""),
                           (opt_2_ifo_ref, "nle-initial vals", "--"))

In [ ]:
# comparing to Vaishali's dashed pink internal squeezed NEMO plot - part 2
a2_lsrc = 319
a2_sqz_gain = 0.12 # 0.065?
a2_sqz_angle = 0
a2_homodyne_angle = 90
a2_Tsrm = 0.12

a2_Tprm = 1 #0.035
#0.35
#0.06318

a2_fmin = 1e2
a2_fmax = 1e4

a2_ifo_no_nle_ref = IFO(0, a2_lsrc, fmin=a2_fmin, fmax=a2_fmax, T_srm=a2_Tsrm, T_prm=a2_Tprm)
a2_ifo_no_nle_lossless = IFO(0, a2_lsrc, fmin=a2_fmin, fmax=a2_fmax, T_srm=a2_Tsrm, T_prm=a2_Tprm)
a2_ifo_sqz = IFO(1, a2_lsrc, fmin=a2_fmin, fmax=a2_fmax, sqz_gain=a2_sqz_gain, sqz_angle=a2_sqz_angle,
                 homodyne_angle=a2_homodyne_angle, T_srm=a2_Tsrm, T_prm=a2_Tprm)
a2_ifo_sqz_lossless = IFO(1, a2_lsrc, fmin=a2_fmin, fmax=a2_fmax, sqz_gain=a2_sqz_gain, sqz_angle=a2_sqz_angle,
                 homodyne_angle=a2_homodyne_angle, T_srm=a2_Tsrm, T_prm=a2_Tprm)

# checking power without squeezer
# check_tuning(a2_ifo_no_nle_ref.kat, power_check=True)

# losses in ppm
Lprm = 0
Litm = 20 #
Letm = Litm

adjust_losses(a2_ifo_no_nle_ref.kat, Loss_itm=Litm*1e-6, Loss_etm=Letm*1e-6, Loss_prm=Lprm*1e-6, Loss_srm=0)
# check_tuning(a2_ifo_no_nle_ref.kat, power_check=True)
a2_ifo_no_nle_ref.kat.laser.P = 5350 #90 #125*14
check_tuning(a2_ifo_no_nle_ref.kat, power_check=True)

adjust_losses(a2_ifo_sqz.kat, Loss_itm=Litm*1e-6, Loss_etm=Letm*1e-6, Loss_prm=Lprm*1e-6, Loss_srm=0)
# check_tuning(a2_ifo_sqz.kat, power_check=True)
a2_ifo_sqz.kat.laser.P = 5350 #125*14
check_tuning(a2_ifo_sqz.kat, power_check=True)

adjust_losses(a2_ifo_sqz_lossless.kat, Loss_itm=0*1e-6, Loss_etm=0*1e-6, Loss_prm=0*1e-6, Loss_srm=0)
a2_ifo_sqz_lossless.kat.laser.P = 5350 #125*14
check_tuning(a2_ifo_sqz_lossless.kat, power_check=True)

adjust_losses(a2_ifo_no_nle_lossless.kat, Loss_itm=0*1e-6, Loss_etm=0*1e-6, Loss_prm=0*1e-6, Loss_srm=0)
a2_ifo_no_nle_lossless.kat.laser.P = 5350 #125*14
check_tuning(a2_ifo_no_nle_lossless.kat, power_check=True)

In [ ]:
analytics_arm_power = 758.9262856692113
analytics_BS_power = 5.35

analytics_arm_power/(analytics_BS_power/2)

In [ ]:
out_i = differential_transfer(a2_ifo_no_nle_ref)
noise_i = out_i['qhd1']
signal_i = out_i['hd1']
plot_x_i = out_i.x

out_ii = differential_transfer(a2_ifo_sqz)
noise_ii = out_ii['qhd1']
signal_ii = out_ii['hd1']
# plot_x_ii = out_ii.x

out_iii = differential_transfer(a2_ifo_sqz_lossless)
noise_iii = out_iii['qhd1']
signal_iii = out_iii['hd1']

out_iv = differential_transfer(a2_ifo_no_nle_lossless)
noise_iv = out_iv['qhd1']
signal_iv = out_iv['hd1']


In [ ]:
# Analytics: taken from Vaishali

# Reduced Planck's constant
hbar = 1.0545718e-34
# Speed of light
c = 299792458.0
# Boltzmann constant
kb = 1.38064852e-23
# Carrier wavelength
lam = 1.064e-6
# Test mass masses
m = 40#np.inf
# Carrier frequency
f0 = c/lam
# Carrier angular frequency
w0 = 2.0*np.pi*f0
# Number of frequency points
N = 500
# Frequency, max and min. 
F_min = 10
F_max = 1e5

# Frequency array
Fs = np.logspace(np.log10(F_min), np.log10(F_max), N)
# Angular frequency
Ws = 2.0*np.pi*Fs
# SQL
#SQL_h = np.sqrt((8.0*hbar)/(m*Ws**2*L**2))
####################################
# IFO PARAMETERS
####################################
# Beam splitter power
P_BS = 1300.0

# Lengths
# -------
L = 3994.469    # Arms
Lsrd = 56.01031  # SR

# Tunings
# --------
# Arm cavity mirrors
T_ITM = 0.014
R_ITM = 1 - T_ITM
t_itm = np.sqrt(T_ITM)
r_itm = np.sqrt(R_ITM)

# SRC mirrors
T_SRM = 0.2
R_SRM = 1 - T_SRM
t_srm = np.sqrt(T_SRM)
r_srm = np.sqrt(R_SRM)

# Other stuff
# -----------
# SRM tuning
phid = 0
# Free spectral range
FSR = c/(2.0*L)
# Arm bandwidth (angular frequency)
# gamma = (T_ITM*c)/(4.0*L)
gamma = 2.0*np.pi*(2.0*FSR/np.pi)*np.arcsin( (1.0-np.sqrt(1.0-T_ITM))/(2.0*(1.0-T_ITM)**(1.0/4.0) ) )/2.0

#gamma = 2*np.pi*66
#tau_arm  = L/c # Storage time in arm cavities
#tau_src = Lsr/c #Storage time in SRC
# Squeezing
# ------------------
Loss = 35e-6 #Internal losses in SRC
Loss2 = 5000e-6  # 0.5% loss in readout is 5000e-6 -- should correspond to about an order of magnitude
Loss3 = 50e-6 # squeezing Injection losses
rd = 0
phi_sqz = 0
rd2 = 0 #in dB
# ------------------

# AC matrix
# This matrix relates the DC carrier fields (a to k) to each other

M = np.zeros((27,27), dtype=complex)

np.fill_diagonal(M, 1)

# Indices for matrix
# DC Fields
# The propagation of the carrier field can is calculated independently from the sideband propagation
# because the carrier is not affected by the sidebands. This is done by populating a
# matrix, M, that relates the fields to each other. Fields a through k are assigned a number 0 through 18 
# that refer to their position in the matrix. The set of input-output relations for the carrier fields are

# compact assignment (added by James)
(a1, a2, b1, b2, c1, c2, d1, d2, e1, e2, f1, f2, g1, g2,
 l1, l2, k1, k2, lv1, lv2, v1, v2, lb1, lb2, n1, n2) = range(0, 26)
# GW signal
h = 26
def min_jet(lam=lam, ma=m, T_ITMs=T_ITM, P_BSs=P_BS, IS = Loss, EL = Loss2,
            IL = Loss3, T_SRMs = T_SRM, r2 = rd2, r = rd, L1=L, Lsr = Lsrd , phi = phid):
    
    if not isinstance(T_SRMs, np.ndarray):
        T_SRMs = np.array([T_SRMs])
    
    # Creating vectors to store quantum noise
    QN1 = np.zeros([len(Fs),len(T_SRMs)])
    QN2 = np.zeros([len(Fs),len(T_SRMs)])
    
    # Creating vectors to store interferometer response
    R_h1 = np.zeros([len(Fs),len(T_SRMs)])
    R_h2 = np.zeros([len(Fs),len(T_SRMs)])
   
    # Carrier frequency
    f0 = c/lam
    # Carrier angular frequency
    w0 = 2.0*np.pi*f0
    
    # ITM transmission
    T_ITMs = T_ITM
    R_ITM = 1 - T_ITM
    t_itm = np.sqrt(T_ITM)
    r_itm = np.sqrt(R_ITM)
    
    #gamma
    gamma = 2.0*np.pi*(2.0*FSR/np.pi)*np.arcsin((1.0-np.sqrt(1.0-T_ITMs))/(2.0*(1.0-T_ITMs)**(1.0/4.0)))/2.0
    
    for k,T_SRM in enumerate(T_SRMs):
        R_SRM = 1.0 - T_SRM;   # SRM Reflectivity
        t_srm = np.sqrt(T_SRM)
        r_srm = np.sqrt(R_SRM)

        for i,W in enumerate(Ws):

            # Locking/probe field relations

            # IFO input/output relation
            # is this only an approximation?
            K = (8.0*w0*P_BSs)/(ma*L1**2*W**2*(gamma**2 + W**2 ))

            alpha = np.sqrt((2*w0*P_BSs)/((gamma**2 + W**2)*hbar))

            beta = np.arctan(W/gamma) 

            M[d1,c1] = np.exp(1j * 2.0 * beta) * 1
            M[d1,c2] = np.exp(1j  * 2.0 * beta) * 0
            M[d2,c1] = np.exp(1j  * 2.0 * beta) * -K
            M[d2,c2] = np.exp(1j  * 2.0 * beta) * 1

            M[d2,h] = np.exp(1j * beta) * alpha

            # SRM refl/trans relation
            M[b1,a1]  = np.sqrt(R_SRM) # b = rs*a + ts*f
            M[b1,f1]  = np.sqrt(T_SRM)
            M[b2,a2]  = np.sqrt(R_SRM)
            M[b2,f2]  = np.sqrt(T_SRM)

            M[e1,a1]  = np.sqrt(T_SRM) # e = ts*a - rs*d
            M[e1,f1]  = -np.sqrt(R_SRM)
            M[e2,a2]  = np.sqrt(T_SRM)
            M[e2,f2]  = -np.sqrt(R_SRM)

            # phase shift and rotation from free space 
            M[g1,k1]  = np.exp(1j*W*Lsr/c) * np.cos(phi)
            M[g1,k2]  = np.exp(1j*W*Lsr/c) * np.sin(phi)*-1
            M[g2,k1]  = np.exp(1j*W*Lsr/c) * np.sin(phi)
            M[g2,k2]  = np.exp(1j*W*Lsr/c) * np.cos(phi)

            M[f1,d1]  = np.exp(1j*W*Lsr/c)*np.cos(phi)
            M[f1,d2] = np.exp(1j*W*Lsr/c)*np.sin(phi)*-1
            M[f2,d1] = np.exp(1j*W*Lsr/c)*np.sin(phi)
            M[f2,d2] = np.exp(1j*W*Lsr/c)*np.cos(phi)

            # Coupling internal loss port
            M[k1,e1] = np.sqrt(1-IS);
            M[k1,l1] = np.sqrt(IS)
            M[k2,e2] = np.sqrt(1-IS)
            M[k2,l2] = np.sqrt(IS)

            # Squeezer relation
            M[c1,g1] = np.cosh(r) + np.sinh(r)*np.cos(2*phi_sqz)
            M[c1,g2] = np.sinh(r)*np.sin(2*phi_sqz)
            M[c2,g1] = np.sinh(r)*np.sin(2*phi_sqz)
            M[c2,g2] = np.cosh(r) - np.sinh(r)*np.cos(2*phi_sqz)   
            
            # Coupling external squeezing injection losses  
            M[a1,v1] = np.sqrt(1-IL);
            M[a1,lv1] = np.sqrt(IL)
            M[a2,v2] = np.sqrt(1-IL)
            M[a2,lv2] = np.sqrt(IL)

            # Coupling readout losses
            M[n1,b1] = np.sqrt(1-EL)
            M[n1,lb1] = np.sqrt(EL)
            M[n2,b2] = np.sqrt(1-EL)
            M[n2,lb2] = np.sqrt(EL)
            
            #inverse matrix

            #Inverting the matrix M gives the propagating fields in terms of the input fields. 
            #By inverting the matrix, we now have access to information about the amplitude 
            #and the phase of the fields at every location inside and outside the ifo.

            inM = np.linalg.inv(M)

            # Quantum noise
            QN1[i,k] = np.sqrt(((np.abs(inM[n1,v1])**2) * (10**(r2/10))) +
                               (np.abs(inM[n1,v2]))**2 * (10**(r2/10)) + 
                                np.abs(inM[n1,lb1])**2 + np.abs(inM[n1,lb2])**2 + 
                              np.abs(inM[n1,l1])**2 + np.abs(inM[n1,l2])**2 +
                               np.abs(inM[n1,lv1])**2 + np.abs(inM[n1,lv2])**2)

            QN2[i,k] = np.sqrt(((np.abs(inM[n2,v1])**2) * (10**(r2/10))) +
                               ((np.abs(inM[n2,v2])**2) * (10**(-r2/10))) + 
                                np.abs(inM[n2,lb1])**2 + np.abs(inM[n2,lb2])**2 + 
                              np.abs(inM[n2,l1])**2 + np.abs(inM[n2,l2])**2 +
                               np.abs(inM[n2,lv1])**2 + np.abs(inM[n2,lv2])**2)


            # GW transfer function         
            R_h1[i,k] = np.abs(inM[n1,h])
            R_h2[i,k] = np.abs(inM[n2,h])


            # Strain sensitivity
            #S1[z] = QN1[z]/R_h1[z];
            #S2[z] = QN2[z]/R_h2[z];


    S_1 = QN1/R_h1
    S_2 = QN2/R_h2
    
    return [QN1, QN2], [R_h1, R_h2], [S_1, S_2]

# no extenal squeezing
QN_sn71_i, R_sn71_i, S_sn71_i =       min_jet(lam,m,T_ITM,5.35e3,Loss,Loss2,Loss3,0.12,0,0.06,L,319,phid)
QN_sn71_ii, R_sn71_ii, S_sn71_ii =    min_jet(lam,m,T_ITM,5.35e3,0,0,0,0.12,0,0.06,L,319,phid)
QN_sn71_iii, R_sn71_iii, S_sn71_iii = min_jet(lam,m,T_ITM,5.35e3,0,0,0,0.12,0,0,L,319,phid)
QN_sn71_iv, R_sn71_iv, S_sn71_iv =    min_jet(lam,m,T_ITM,5.35e3,Loss,Loss2,Loss3,0.12,0,0,L,319,phid)

In [ ]:
fig1, ax = plt.subplots(figsize=(8,6), dpi=300)
 
ax.loglog(plot_x_i, noise_iv/signal_iv, '-', label="simulation: no losses, no sqz")
# ax.loglog(plot_x_i, noise_i/signal_i, '|', label="simulation: 20ppm losses, no sqz")
ax.loglog(plot_x_i, noise_iii/signal_iii, '--', label="simulation: no losses, sqz")
# ax.loglog(plot_x_i, noise_ii/signal_ii, '--', label="simulation: 20ppm losses, sqz")

ax.loglog(Fs,  S_sn71_iii[1],'_' , alpha=1.0,label='analytics: no losses, no sqz')
# ax.loglog(Fs,  S_sn71_iv[1],'--' , alpha=1.0,label='analytics: losses, no sqz')
ax.loglog(Fs,  S_sn71_ii[1],'--' , alpha=1.0,label='analytics: no losses, sqz')
# ax.loglog(Fs,  S_sn71_i[1],'--' ,alpha=1.0,label='analytics: losses, sqz')

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_axisbelow(True)
ax.grid(b=True,which='minor', color='k', linestyle='-')
ax.legend(loc='upper left')#,fontsize = 'x-small')#,ncol=2)
# ax.set_xlim(1e2,1e4)
# ax.set_ylim(3E-25,1E-22)
ax.set_xlim(5e2,5e3)
ax.set_ylim(3E-25,2e-23)
ax.set_xlabel('Frequency [Hz]',fontsize=12)
ax.set_ylabel("Sensitivity [1/$\sqrt{Hz}$]",fontsize=12)
#plt.savefig('comparison_2km_bounded.eps')
plt.show(fig1)

fig1.savefig("sqz_aLIGO_analytics_v_simulation.pdf")

print("peak sensitivity of {0:.3g} Hz^-0.5 obtained at {1:.5g} Hz".
      format(np.min(S_sn71_ii[1]), Fs[np.argmin(S_sn71_ii[1])]))
# print("peak sensitivity of {0:.3g} Hz^-0.5 obtained at {1:.5g} Hz".
#       format(np.min(S_sn71_ii[1]), Fs[np.argmin(S_sn71_ii[1])]))
# print("peak sensitivity of {0:.3g} Hz^-0.5 obtained at {1:.5g} Hz".
#       format(np.min(S_sn71_iii[1]), Fs[np.argmin(S_sn71_iii[1])]))
# print("peak sensitivity of {0:.3g} Hz^-0.5 obtained at {1:.5g} Hz".
#       format(np.min(noise_i/signal_i), plot_x_i[np.argmin(noise_i/signal_i)]))
print("peak sensitivity of {0:.3g} Hz^-0.5 obtained at {1:.5g} Hz".
      format(np.min(noise_iii/signal_iii), plot_x_i[np.argmin(noise_iii/signal_iii)]))


In [ ]:
8.92/3.95